In [12]:
import networkx as nx
from scipy.optimize import linprog
from sympy import Matrix, pprint, floor, Point, zeros
from linkages import *

def graph_to_matrix(G, pins={}):
    if type(pins) is Point: pins = {pins}
    M = Matrix()
    for edge in G.edges.data():
        edge_from, edge_to, edge_att = edge
        row = []
        t = -1 if edge_att[TYPE] == "strut" else 1
        delta_x,delta_y = t*(edge_from.x - edge_to.x), t*(edge_from.y - edge_to.y)
        for vertex in G.nodes:
            if vertex == edge_from and vertex not in pins:
                row.extend([delta_x,delta_y])
            elif vertex == edge_to and vertex not in pins:
                row.extend([-delta_x,-delta_y])
            else:
                row.extend([0,0])
        M = Matrix([M,row])
        if edge_att[TYPE] == "bar": M = Matrix([M,M.row(-1)*-1])
    return M

def set_edge_type(G,edge,val):
    a,b = edge
    graph[list(G.nodes)[a]][list(G.nodes)[b]][TYPE] = val
    
def check_rigidity(A): 
    b = zeros(A.rows,1)
    for i in range(0,A.cols):
        if A.col(i) == b: continue
        c = zeros(1,A.cols)
        c[i] = -1
        res = linprog(c, A_ub=A,b_ub=b, method="simplex")
        # if res[‘status] is 0 the optimization converged
        # look through the results in res[‘x’]
        # if any value is larger than 0 return “inf motion of value”
        # if status is 3 return “equation unbound—motion possible”
    # if nothing returned so far, no motion return “rigid”    
    
graph = parallel_4_bar(True) 
set_edge_type(graph,(0,2),"cable")
set_edge_type(graph,(1,3),"cable")

nodes = list(graph.nodes)
A = graph_to_matrix(graph, {nodes[0], nodes[1]})
pprint(A)
print(check_rigidity(A))


⎡0  0  0  0  0   0   0    0 ⎤
⎢                           ⎥
⎢0  0  0  0  5   2   0    0 ⎥
⎢                           ⎥
⎢0  0  0  0  0   0   -5  -10⎥
⎢                           ⎥
⎢0  0  0  0  -5  2   0    0 ⎥
⎢                           ⎥
⎢0  0  0  0  0   0   5   -10⎥
⎢                           ⎥
⎣0  0  0  0  0   -8  0    8 ⎦
the tensegrity is rigid
